### 01. 라이브러리 및 데이터 호출

In [75]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
import matplotlib.pyplot as plt

You should consider upgrading via the 'c:\users\nt930qcg\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached tensorflow_io_gcs_filesystem-0.31.0-cp39-cp39-win_amd64.whl (1.5 MB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached requests-2.31.0-py3-none-any.whl (62 kB)


In [69]:
train = pd.read_csv('data/train.csv')
submission = pd.read_csv('data/submission.csv')

--------------------------

### 02. 데이터 기초 정보 확인

In [70]:
# 데이터 처음,끝부분 확인
head = train.head(5)
tail = train.head(5)

# Columns 확인
columns = train.columns

# Data Shape 확인
shape = train.shape

# 통계값 확인
describe = train.describe()

# Columns Type 확인
dtypes = train.dtypes

-----------------------

### 03. 데이터 전처리

#### 3-1. 결측치 확인
###### [1] 결측치와 중복되는 행이 모두 없음을 확인할 수 있다
###### [2] 따라서 결측치 처리에 대한 부분이 불필요

In [71]:
# 컬럼별 결측치 파악
null = train.isna().sum()

# 중복되는 행 파악
duplicated = train[train.duplicated()]

#### 3-2. 시간 데이터 전처리
###### [1] 시간 기반 데이터는 Time 컬럼을 인덱스로 설정하면 시간 시계열 분석에 유용

In [72]:
# 01. time 컬럼을 datetime 형식으로 변환
train['Time'] = pd.to_datetime(train['Time'])

# 02. 년, 월, 일, 시, 분, 초 등의 특성 추출
train['year'] = train['Time'].dt.year
train['month'] = train['Time'].dt.month
train['day'] = train['Time'].dt.day
train['hour'] = train['Time'].dt.hour
train['minute'] = train['Time'].dt.minute
train['second'] = train['Time'].dt.second

# 03. 요일 추출 (0=월요일, 6=일요일)
train['weekday'] = train['Time'].dt.weekday

# 04. 주차 추출
train['week'] = train['Time'].dt.isocalendar().week


In [76]:
# 데이터 읽기
train = pd.read_csv('data/train.csv')
submission = pd.read_csv('data/submission.csv')

# 01. time 컬럼을 datetime 형식으로 변환
train['Time'] = pd.to_datetime(train['Time'])

# 02. 년, 월, 일, 시, 분, 초 등의 특성 추출
train['year'] = train['Time'].dt.year
train['month'] = train['Time'].dt.month
train['day'] = train['Time'].dt.day
train['hour'] = train['Time'].dt.hour
train['minute'] = train['Time'].dt.minute
train['second'] = train['Time'].dt.second

# 03. 요일 추출 (0=월요일, 6=일요일)
train['weekday'] = train['Time'].dt.weekday

# 04. 주차 추출
train['week'] = train['Time'].dt.isocalendar().week

# 비트코인 가격 예측을 위한 종가(Close) 컬럼 선택
data = train[['Time', 'Close']].set_index('Time')

# 데이터 정규화
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# 학습 데이터 준비
def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset) - look_back - 1):
        a = dataset[i:(i + look_back), 0]
        X.append(a)
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)

look_back = 60
X, Y = create_dataset(scaled_data, look_back)

# 데이터 셰이프 변환 [samples, time steps, features]
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

# LSTM 모델 구축
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(look_back, 1)))
model.add(LSTM(50))
model.add(Dense(1))

# 모델 컴파일
model.compile(optimizer='adam', loss='mean_squared_error')

# 모델 학습
model.fit(X, Y, epochs=20, batch_size=32)

# 예측
predicted = model.predict(X)
predicted = scaler.inverse_transform(predicted)

# 결과 시각화
plt.plot(data.index[look_back+1:], data.values[look_back+1:], label='Actual')
plt.plot(data.index[look_back+1:], predicted, label='Predicted')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()


Epoch 1/20


c:\Users\NT930QCG\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1704/1704 ━━━━━━━━━━━━━━━━━━━━ 102s 58ms/step - loss: 0.0020
Epoch 2/20
1704/1704 ━━━━━━━━━━━━━━━━━━━━ 105s 62ms/step - loss: 4.9588e-05
Epoch 3/20
1704/1704 ━━━━━━━━━━━━━━━━━━━━ 121s 71ms/step - loss: 3.4630e-05
Epoch 4/20
1704/1704 ━━━━━━━━━━━━━━━━━━━━ 116s 68ms/step - loss: 2.7682e-05
Epoch 5/20
1704/1704 ━━━━━━━━━━━━━━━━━━━━ 96s 56ms/step - loss: 2.4808e-05
Epoch 6/20
1704/1704 ━━━━━━━━━━━━━━━━━━━━ 97s 57ms/step - loss: 1.6752e-05
Epoch 7/20
1704/1704 ━━━━━━━━━━━━━━━━━━━━ 109s 37ms/step - loss: 1.4281e-05
Epoch 8/20
1704/1704 ━━━━━━━━━━━━━━━━━━━━ 93s 55ms/step - loss: 1.4059e-05
Epoch 9/20
1704/1704 ━━━━━━━━━━━━━━━━━━━━ 96s 56ms/step - loss: 1.4475e-05
Epoch 10/20
1704/1704 ━━━━━━━━━━━━━━━━━━━━ 90s 53ms/step - loss: 1.4402e-05
Epoch 11/20
1704/1704 ━━━━━━━━━━━━━━━━━━━━ 75s 44ms/step - loss: 1.4760e-05
Epoch 12/20
1704/1704 ━━━━━━━━━━━━━━━━━━━━ 75s 44ms/step - loss: 1.3265e-05
Epoch 13/20
1704/1704 ━━━━━━━━━━━━━━━━━━━━ 107s 63ms/step - loss: 1.3616e-05
Epoch 14/20
1704/1704 ━━━━━━━━

내일 실행해볼 전체 코드

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# 데이터 읽기
train = pd.read_csv('data/train.csv')
submission = pd.read_csv('data/submission.csv')

# 01. time 컬럼을 datetime 형식으로 변환
train['Time'] = pd.to_datetime(train['Time'])
submission['Time'] = pd.to_datetime(submission['Time'])

# 02. 년, 월, 일, 시, 분, 초 등의 특성 추출
train['year'] = train['Time'].dt.year
train['month'] = train['Time'].dt.month
train['day'] = train['Time'].dt.day
train['hour'] = train['Time'].dt.hour
train['minute'] = train['Time'].dt.minute
train['second'] = train['Time'].dt.second

submission['year'] = submission['Time'].dt.year
submission['month'] = submission['Time'].dt.month
submission['day'] = submission['Time'].dt.day
submission['hour'] = submission['Time'].dt.hour
submission['minute'] = submission['Time'].dt.minute
submission['second'] = submission['Time'].dt.second

# 03. 요일 추출 (0=월요일, 6=일요일)
train['weekday'] = train['Time'].dt.weekday
submission['weekday'] = submission['Time'].dt.weekday

# 04. 주차 추출
train['week'] = train['Time'].dt.isocalendar().week
submission['week'] = submission['Time'].dt.isocalendar().week

# 비트코인 가격 예측을 위한 종가(Close) 컬럼 선택
data = train[['Time', 'Close']].set_index('Time')
submission_data = submission.set_index('Time')

# 데이터 정규화
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# 학습 데이터 준비
def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset) - look_back - 1):
        a = dataset[i:(i + look_back), 0]
        X.append(a)
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)

look_back = 60
X, Y = create_dataset(scaled_data, look_back)

# 데이터 셰이프 변환 [samples, time steps, features]
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

# LSTM 모델 구축
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(look_back, 1)))
model.add(LSTM(50))
model.add(Dense(1))

# 모델 컴파일
model.compile(optimizer='adam', loss='mean_squared_error')

# 모델 학습
model.fit(X, Y, epochs=20, batch_size=32)

# 학습된 모델을 사용하여 submission 데이터 예측
submission_scaled_data = scaler.transform(submission_data)
submission_X = []
for i in range(len(submission_scaled_data) - look_back):
    submission_X.append(submission_scaled_data[i:(i + look_back), 0])

submission_X = np.array(submission_X)
submission_X = np.reshape(submission_X, (submission_X.shape[0], submission_X.shape[1], 1))

# 예측
submission_predict = model.predict(submission_X)
submission_predict = scaler.inverse_transform(submission_predict)

# 결과를 DataFrame으로 변환
submission_result = pd.DataFrame(submission_predict, columns=['Predicted_Close'])
submission_result['Time'] = submission['Time'][look_back:].values

# 결과 저장
submission_result.to_csv('submission_predictions.csv', index=False)

# 결과 시각화 (선택 사항)
plt.figure(figsize=(12, 6))
plt.plot(submission['Time'][look_back:], submission_predict, label='Predicted')
plt.xlabel('Date')
plt.ylabel('Predicted Close Price')
plt.legend()
plt.show()
